## Create the Test Data for four scenerios

In [ ]:
import os
os.chdir('../..')
from trips.config import *
from trips.weibull_trips import main
import subprocess

In [ ]:
scene_src_probabilities = {
    "High Traffic Scenerio": [0.25, 0.25, 0.25, 0.25],
    "Low Traffic Scenerio": [0.25, 0.25, 0.25, 0.25],
    "NS-Traffic Scenerio": [0.45, 0.05, 0.45, 0.05],
    "EW-Traffic Scenerio": [0.05, 0.45, 0.05, 0.45],
}

scene_number_cars = {
    "High Traffic Scenerio": 1000,
    "Low Traffic Scenerio": 150,
    "NS-Traffic Scenerio": 500,
    "EW-Traffic Scenerio": 500,
}


if not os.path.exists("test"):
    os.makedirs("test")
    
for scene in scene_src_probabilities:
    main(
        src_nodes,
        dst_nodes,
        scene_src_probabilities[scene],
        turn_prob,
        emergency_probs,
        shape,
        scene_number_cars[scene],
        total_timesteps,
    )
    subprocess.run(
        [
            "duarouter",
            "-n",
            "./network_details/intersection.net.xml",  # Input network file
            "-t",
            "trips.trips.xml",  # Input trips file
            "-o",
            f"test/intersection_{scene}.rou.xml",  # Output routes file
        ]
    )
os.remove("trips.trips.xml")


route_files = [f"test/intersection_{scene}.rou.xml" for scene in scene_src_probabilities]
        

## Test round robin algorithm on the scenerios

In [4]:
from environment.environment import MultiRouteSumoEnvironment

In [ ]:
for i in range(4):
    env = MultiRouteSumoEnvironment(
        net_file="network_details/intersection.net.xml",
        route_files=[route_files[i]],
        out_csv_name=f'outputs/test/{i}/RoundRobin',
        min_green=25,
        yellow_time=5,
        delta_time=30,
        use_gui=False,
        num_seconds=5400
    )

    obs, info = env.reset()
    done = False

    action_index = 0  
    while not done:
        action = action_index % 4
        
        next_obs, reward, terminated, truncated, info = env.step(action)
        done = terminated or truncated

        action_index += 1

    obs, info = env.reset()
    env.close()

 Retrying in 1 seconds
Step #0.00 (0ms ?*RT. ?UPS, TraCI: 12ms, vehicles TOT 0 ACT 0 BUF 0)                     
 Retrying in 1 seconds
Step #5400.00 (0ms ?*RT. ?UPS, TraCI: 6ms, vehicles TOT 1000 ACT 1 BUF 0)                 tep #4500.00 (0ms ?*RT. ?UPS, TraCI: 9ms, vehicles TOT 996 ACT 1 BUF 0)                  
 Retrying in 1 seconds
Step #0.00 (0ms ?*RT. ?UPS, TraCI: 12ms, vehicles TOT 0 ACT 0 BUF 0)                     
 Retrying in 1 seconds
Step #0.00 (0ms ?*RT. ?UPS, TraCI: 4ms, vehicles TOT 0 ACT 0 BUF 0)                      
 Retrying in 1 seconds
Step #5400.00 (0ms ?*RT. ?UPS, TraCI: 4ms, vehicles TOT 150 ACT 0 BUF 0)                  tep #4500.00 (0ms ?*RT. ?UPS, TraCI: 5ms, vehicles TOT 146 ACT 0 BUF 0)                  
 Retrying in 1 seconds
Step #0.00 (0ms ?*RT. ?UPS, TraCI: 29ms, vehicles TOT 0 ACT 0 BUF 0)                     
 Retrying in 1 seconds
Step #0.00 (0ms ?*RT. ?UPS, TraCI: 10ms, vehicles TOT 0 ACT 0 BUF 0)                     
 Retrying in 1 seconds
Step #

## Test trained DQN Policy on the scenerios

In [ ]:
from stable_baselines3 import DQN

model = DQN.load("agents/dqn_multi_scene")

for i in range(4):
    env = MultiRouteSumoEnvironment(
        net_file="network_details/intersection.net.xml",
        route_files=[route_files[i]],
        out_csv_name=f'outputs/test/{i}/DQN',
        min_green=5,
        yellow_time=5,
        delta_time=10,
        use_gui=False,
        num_seconds=5400
    )
    
    obs, info = env.reset()
    done = False

    while not done:
        action, _states = model.predict(obs, deterministic=True)
        obs, reward, terminated, truncated, info = env.step(action)
        done = terminated or truncated

    obs, info = env.reset()
    env.close()

/home/saksham/miniconda3/envs/sumo-rl/lib/python3.12/site-packages/stable_baselines3/common/save_util.py:167: UserWarning: Could not deserialize object lr_schedule. Consider using `custom_objects` argument to replace this object.
Exception: code() argument 13 must be str, not int
  warnings.warn(
/home/saksham/miniconda3/envs/sumo-rl/lib/python3.12/site-packages/stable_baselines3/common/save_util.py:167: UserWarning: Could not deserialize object exploration_schedule. Consider using `custom_objects` argument to replace this object.
Exception: code() argument 13 must be str, not int
  warnings.warn(


 Retrying in 1 seconds
Step #0.00 (0ms ?*RT. ?UPS, TraCI: 11ms, vehicles TOT 0 ACT 0 BUF 0)                     
 Retrying in 1 seconds
Step #5400.00 (0ms ?*RT. ?UPS, TraCI: 6ms, vehicles TOT 1000 ACT 0 BUF 0)                 tep #4500.00 (0ms ?*RT. ?UPS, TraCI: 8ms, vehicles TOT 996 ACT 1 BUF 0)                  
 Retrying in 1 seconds
Step #0.00 (0ms ?*RT. ?UPS, TraCI: 10ms, vehicles TOT 0 ACT 0 BUF 0)                     
 Retrying in 1 seconds
Step #0.00 (0ms ?*RT. ?UPS, TraCI: 12ms, vehicles TOT 0 ACT 0 BUF 0)                     
 Retrying in 1 seconds
Step #5400.00 (0ms ?*RT. ?UPS, TraCI: 5ms, vehicles TOT 150 ACT 0 BUF 0)                  tep #4500.00 (0ms ?*RT. ?UPS, TraCI: 7ms, vehicles TOT 146 ACT 0 BUF 0)                  
 Retrying in 1 seconds
Step #0.00 (0ms ?*RT. ?UPS, TraCI: 11ms, vehicles TOT 0 ACT 0 BUF 0)                     
 Retrying in 1 seconds
Step #0.00 (0ms ?*RT. ?UPS, TraCI: 7ms, vehicles TOT 0 ACT 0 BUF 0)                      
 Retrying in 1 seconds
Step #

## Test the trained A2C algorithm on the scenerios

In [ ]:
from stable_baselines3 import A2C
from stable_baselines3.common.vec_env import DummyVecEnv

model = A2C.load('agents/a2c_multi_scene')

for i in range(4):
    def make_env():
        return MultiRouteSumoEnvironment(
            net_file="network_details/intersection.net.xml",
            route_files=route_files,
            out_csv_name=f'outputs/test/{i}/A2C',
            min_green=5,
            yellow_time=5,
            delta_time=10,
            use_gui=False,
            num_seconds=5400
        )
        
    test_env = DummyVecEnv([make_env])

    obs = test_env.reset()
    done = False
    while not done:
        action, _states = model.predict(obs, deterministic=True)
        obs, reward, done, info = test_env.step(action)


/home/saksham/miniconda3/envs/sumo-rl/lib/python3.12/site-packages/stable_baselines3/common/save_util.py:167: UserWarning: Could not deserialize object lr_schedule. Consider using `custom_objects` argument to replace this object.
Exception: code() argument 13 must be str, not int
  warnings.warn(


 Retrying in 1 seconds
Step #0.00 (0ms ?*RT. ?UPS, TraCI: 11ms, vehicles TOT 0 ACT 0 BUF 0)                     
 Retrying in 1 seconds
Step #5400.00 (0ms ?*RT. ?UPS, TraCI: 6ms, vehicles TOT 1000 ACT 0 BUF 0)                 tep #4500.00 (0ms ?*RT. ?UPS, TraCI: 9ms, vehicles TOT 996 ACT 1 BUF 0)                  
 Retrying in 1 seconds
 Retrying in 1 seconds
Step #0.00 (0ms ?*RT. ?UPS, TraCI: 11ms, vehicles TOT 0 ACT 0 BUF 0)                     
 Retrying in 1 seconds
Step #5400.00 (0ms ?*RT. ?UPS, TraCI: 5ms, vehicles TOT 150 ACT 0 BUF 0)                  tep #4500.00 (0ms ?*RT. ?UPS, TraCI: 8ms, vehicles TOT 146 ACT 0 BUF 0)                  
 Retrying in 1 seconds
 Retrying in 1 seconds
Step #0.00 (0ms ?*RT. ?UPS, TraCI: 10ms, vehicles TOT 0 ACT 0 BUF 0)                     
 Retrying in 1 seconds
Step #5400.00 (0ms ?*RT. ?UPS, TraCI: 4ms, vehicles TOT 500 ACT 0 BUF 0)                  tep #4500.00 (0ms ?*RT. ?UPS, TraCI: 11ms, vehicles TOT 496 ACT 1 BUF 0)                 
 Retry